In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

class HTMLTableParser :
    # 하나의 페이지에 여러개의 테이블이 있을 수 있다
    def parse_url(self, url) :
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table('id'), self.parse_html_table(table)) for table in soup.find_all('table')]
    
    def parse_html_table(self, table) :   # 2차원 : 행과 열로 이루어져 있다
        n_columns = 0
        n_rows = 0
        column_names = []
        for row in table.find_all('tr') :
            td_tags = row.find_all('td')
            if len(td_tags)>0 :   # 맨처음이 아니면 행수를 +1
                n_rows += 1
                if n_columns == 0 :   # 맨처음이면 열수를 세라
                    n_columns = len(td_tags)
            th_tags = row.find_all('th')
            if len(th_tags)>0 and len(column_names) == 0 :   # 열수가 있고 열이름이 미정인 경우
                for th in th_tags :
                    column_names.append(th.get_text())   # th태그에서 가져온 이름으로 열이름 부여
                    
        # 열수 만큼 열이름이 존재해야 하는데 그 수가 맞지 않으면 예외처리
        if len(column_names) > 0 and len(column_names) != n_columns :
            raise Exception('컬럼 타이틀 수가 컬럼 수와 다름') 
            
        #열이름이 없으면 index로 줘라
        columns = column_names if len(column_names)>0 else range(0,n_columns)
        
        # pandas : R에서 배운 DataFrame과 같은 역할
        # 열(col)내에서는 동질적, 열(col)간에는 이질적 데이터
        df = pd.DataFrame(columns=columns, index=range(0,n_rows))   # 배열 -> 인덱스, 저장공간 만들기
        
        # 데이터 파싱
        row_marker = 0
        for row in table.find_all('tr') :  
            column_marker = 0;
            columns = row.find_all('td')
            for column in columns :
                df.iloc[row_marker, column_marker] = column.get_text()   # 0행 0열부터 시작
                column_marker += 1
            if len(columns)>0 :
                row_marker += 1
                
        # 숫자는 숫자로 변환
        for col in df :
            try:
                df[col] = df[col].astype(float)   # 주의 : 문자를 숫자로 변환하려 하면 에러 발생
            except ValueError :
                pass
        return df        

In [10]:
import re

# 기업 정보 공지 사이트
date = "2020.09.07"
url_part1 = """http://dart.fss.or.kr/dsac001/search.ax?selectDate="""
url_part2 = """&sort=&series=&mdayCnt=0&currentPage="""
url = url_part1 + date + url_part2

hp = HTMLTableParser()
table = hp.parse_url(url)[0][1]
table.columns

for i in table.columns:
    for j in range(len(table[i])):
        table[i][j] = re.sub('\s','',table[i][j])
table

,시간,공시대상회사,보고서명,제출인,접수일자,비고
0,18:43,아래스,주권매매거래정지기간변경(개선기간부여),코스닥시장본부,2020.09.07,
1,18:36,코오롱티슈진,기타시장안내(상장폐지관련이의신청서접수),코스닥시장본부,2020.09.07,
2,18:33,에이아이비트,주권매매거래정지기간변경(개선기간부여),코스닥시장본부,2020.09.07,
3,18:30,아래스,기타시장안내(개선기간부여안내),코스닥시장본부,2020.09.07,
4,18:29,에이아이비트,기타시장안내(개선기간부여안내),코스닥시장본부,2020.09.07,
...,...,...,...,...,...,...
95,16:38,키움증권,일괄신고추가서류(파생결합증권-주가연계증권),키움증권,2020.09.07,
96,16:38,현대모비스,기업설명회(IR)개최(안내공시),현대모비스,2020.09.07,
97,16:37,현대캐피탈,투자설명서(일괄신고),현대캐피탈,2020.09.07,
98,16:37,SGA임베디드,임시주주총회결과,SGA임베디드,2020.09.07,


In [7]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

class HTMLTableParser :
    # 하나의 페이지에 여러개의 테이블이 있을 수 있다
    def parse_url(self, url) :
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table('id'), self.parse_html_table(table)) for table in soup.find_all('table')]
        # id 테이블 이름[0], table 테이블 내용[1]
        # 1번째 테이블의 테이블 내용 -> [0][1]
        # tag 또는 attribute 기준으로 찾고 -> string으로
    
    def parse_html_table(self, table) :   # 2차원 : 행과 열로 이루어져 있다
        n_columns = 0
        n_rows = 0
        column_names = []
        for row in table.find_all('tr') :
            td_tags = row.find_all('td')
            if len(td_tags)>0 :   # 맨처음이 아니면 행수를 +1
                n_rows += 1
                if n_columns == 0 :   # 맨처음이면 열수를 세라
                    n_columns = len(td_tags)
            th_tags = row.find_all('th')
            if len(th_tags)>0 and len(column_names) == 0 :   # 열수가 있고 열이름이 미정인 경우
                for th in th_tags :
                    column_names.append(th.get_text())   # th태그에서 가져온 이름으로 열이름 부여
                    
        # 열수 만큼 열이름이 존재해야 하는데 그 수가 맞지 않으면 예외처리
        if len(column_names) > 0 and len(column_names) != n_columns :
            raise Exception('컬럼 타이틀 수가 컬럼 수와 다름') 
            
        #열이름이 없으면 index로 줘라
        columns = column_names if len(column_names)>0 else range(0,n_columns)
        
        # pandas : R에서 배운 DataFrame과 같은 역할
        # 열(col)내에서는 동질적, 열(col)간에는 이질적 데이터
        df = pd.DataFrame(columns=columns, index=range(0,n_rows))   # 배열 -> 인덱스, 저장공간 만들기
        
        # 데이터 파싱
        row_marker = 0
        for row in table.find_all('tr') :  
            column_marker = 0;
            columns = row.find_all('td')
            for column in columns :                
                regex2 = re.compile(r'\n')
                re_text = re.sub(regex2, '', column.get_text())   # soul.strings로 가져오면 전체텍스트를 가져오게됨
                regex2 = re.compile(r'\t')
                re_text = re.sub(regex2, '', re_text)
                regex2 = re.compile(r'\r')
                re_text = re.sub(regex2, '', re_text)
                df.iloc[row_marker, column_marker] = re_text   # 0행 0열부터 시작
                #df.iloc[row_marker, column_marker] = column.get_text()   # 0행 0열부터 시작
                column_marker += 1
            if len(columns)>0 :
                row_marker += 1
                
        # 숫자는 숫자로 변환
        for col in df :   # 열 확인, 열 중심으로 데이터 저장
            try:   
                df[col] = df[col].astype(float)   # 주의 : 문자를 숫자로 변환하려 하면 에러 발생
            except ValueError :
                pass
        return df        

In [49]:
from bs4 import BeautifulSoup
import requests
url  = 'http://www.naver.com'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
mr = soup.find('a')
print(len(mr))
print(mr)

1
<a href="#newsstand"><span>뉴스스탠드 바로가기</span></a>


In [45]:
title = soup.find(class_="title elss")
titleList = title.find_all('a')
for list in titleList:
    print(list.get_text())

In [46]:
menu1 = soup.find(id="footer")    
menulist1 = menu1.find_all('a')
for menu in menulist1:
    print(menu.get_text())

공지사항
서비스 전체보기
크리에이터
스몰비즈니스
비즈니스 · 광고
스토어 개설
지역업체 등록
네이버 개발자 센터
오픈 API
오픈소스
네이버 D2
네이버 D2SF
네이버 랩스
  
다운받기
  
바로가기
회사소개
인재채용
제휴제안
이용약관
개인정보처리방침
청소년보호정책
네이버 정책
고객센터
ⓒ NAVER Corp.


In [51]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen("https://www.naver.com")
bsObject = BeautifulSoup(html, "html.parser")

for link in bsObject.find_all('a'):
    print(link.text.strip(), link.get('href'))

뉴스스탠드 바로가기 #newsstand
주제별캐스트 바로가기 #themecast
타임스퀘어 바로가기 #timesquare
쇼핑캐스트 바로가기 #shopcast
로그인 바로가기 #account
브라우저에 사용 흔적을 남기고 싶지 않다면? 네이버 웨일! https://whale.naver.com/details/incognito?=main&wpid=RydDy7
다운로드 http://update.whale.naver.net/downloads/banner/RydDy7/WhaleSetup.exe
NAVER whale https://whale.naver.com/details/incognito?=main&wpid=RydDy7
네이버 /
네이버를 시작페이지로 https://help.naver.com/support/welcomePage/guide.help
쥬니어네이버 https://jr.naver.com
해피빈 https://happybean.naver.com
입력도구 javascript:;
자동완성/최근검색어 펼치기 javascript:;
도움말 보기 https://help.naver.com/support/contents/contents.nhn?serviceNo=606&categoryNo=16658
ON선택됨 #
OFF #
로그인 https://nid.naver.com/nidlogin.login
자세히 https://help.naver.com/support/alias/search/word/word_16.naver
자세히 https://help.naver.com/support/contents/contents.nhn?serviceNo=606&categoryNo=16659
도움말 https://help.naver.com/support/service/main.nhn?serviceNo=606&categoryNo=1987
신고 https://help.naver.com/support/contents/contents.nhn?serviceNo=605&categoryNo=18215
검색어저장 

In [52]:
from urllib.request import urlopen
import urllib.parse
from bs4 import BeautifulSoup
import time
import os

def naverCrawlNews(date=time.strftime('%Y%m%d', time.localtime(time.time())),
                   page_number=2):
    with open("naver뉴스_1.txt", 'w', encoding='utf8') as f:
        f.write(date+"일자 속보\n\n")
    pre_url = "http://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=001&date=" + str(date) +"&page="
    for page_no in range(1, page_number+1):
        pre_url2 = pre_url + str(page_no)
        try:
            url = urlopen(pre_url2)
        except:
            print ("오픈에러")
            return None

        soup = BeautifulSoup(url, 'lxml')

        newslist = soup.find_all('a') #,re.compile('text' or 'title'))
        print ("%d 페이지 크롤링..." % page_no)
        for i in newslist:
            if(i.text != None):
                with open("naver뉴스.txt", 'a', encoding='utf8') as f:
                    f.write(i.text.lstrip() + "\n" + i['href'] + "\n\n")
        print ("저장완료!")
naverCrawlNews()

1 페이지 크롤링...
저장완료!
2 페이지 크롤링...
저장완료!


In [72]:
import json

json_data1 = """{
"color":"red",
"value":"#f00"
}"""

result = json.loads(json_data1)
print (result)
print (result["color"])
print (result["value"]) 

{'color': 'red', 'value': '#f00'}
red
#f00


In [73]:
x = {
  "name": "John",
  "age": 30,
  "married": True,
  "divorced": False,
  "children": ("Ann","Billy"),
  "pets": None,
  "cars": [
    {"model": "BMW 230", "mpg": 27.5},
    {"model": "Ford Edge", "mpg": 24.1}
  ]
}

print(json.dumps(x)) 

{"name": "John", "age": 30, "married": true, "divorced": false, "children": ["Ann", "Billy"], "pets": null, "cars": [{"model": "BMW 230", "mpg": 27.5}, {"model": "Ford Edge", "mpg": 24.1}]}
